In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import scipy.io as sio
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
from nilearn.maskers import NiftiMasker
from nilearn import datasets
from nilearn.connectome import ConnectivityMeasure
from nilearn.maskers import MultiNiftiLabelsMasker
from nilearn import plotting
from statsmodels.stats.multitest import fdrcorrection
from tqdm import tqdm

In [ ]:
colors = np.array([
    "#FF0000",  # Red DMN
    "#00FF00",  # Green VIS
    "#0000FF",  # Blue FP
    "#FFFF00",  # Yellow Reward
    "#FF00FF",  # Magenta DA
    "#00000000",  # nan
    "#00FFFF",  # Cyan VA
    "#800080",  # Purple SN
    "#FFA500",  # Orange CO
    "#008000",  # Dark Green SM Dorsal
    "#800000",  # Maroon SM Lateral
    "#008080",  # Teal AUD
    "#00000000",  # nan
    "#00000000",  # nan
    "#000080",  # Navy ParietoMedial
    "#00000000",  # nan
    "#FFD700",  # Gold Medial Temporal Lobe
    "#00000000",  # nan
])

func_network_affiliation = np.loadtxt("/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/func_networks/masks/ROIs_300inVol_MNI/ROIs_CommunityAffiliation.txt").astype(int)
coordinates = np.loadtxt("/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/func_networks/masks/ROIs_300inVol_MNI/coords_300.csv", delimiter=",", dtype=float)
connectivity = sio.loadmat(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/results/connectivity/300/connectivity_2.mat")["C_net_2"]

network_names = {5: "DA", 17: "MTL", 1: "DMN", 4: "Rew", 8: "SN", 3: "FP", 10: "SMD", 11: "SML", 7: "VA", 9: "CO", 2: "VIS", 12: "AUD", 15: "PM"}

In [ ]:
def calc_roi_stats(csv_path):
    df = pd.read_csv(csv_path)
    df['roi'] = df['roi'].astype('category')
    df['session'] = df['session'].astype('category')
    
    md = smf.mixedlm("difference ~ roi - 1", df, groups=df["session"])
    mdf = md.fit()
    mdf.summary()

    all_data = pd.DataFrame({"pvalues": mdf.pvalues[:-1], "tvalues": mdf.tvalues[:-1], "roi": np.arange(1,300+1)})
    significant_data = all_data[all_data["pvalues"] < 0.05]
    
    significant_positive = significant_data[significant_data["tvalues"] > 0]
    significant_negative = significant_data[significant_data["tvalues"] < 0]

    rois_positive = np.array(significant_positive["roi"].tolist())
    rois_negative = np.array(significant_negative["roi"].tolist())

    corr_positive = np.zeros((len(rois_positive), len(rois_positive)))
    corr_negative = np.zeros((len(rois_negative), len(rois_negative)))

    if len(significant_positive) == 0:
        warnings.warn("Warning: No significant positive ROIs found")
    if len(significant_negative) == 0:
        warnings.warn("Warning: No significant negative ROIs found")

    p_values = np.array(all_data["pvalues"].tolist())
    h, pcorregido = fdrcorrection(p_values, alpha=0.05, method="indep", is_sorted=False)
    indices = np.where(h)[0]
    new_data = all_data.iloc[indices]
    new_positive = np.array(new_data[new_data["tvalues"] > 0]["roi"].tolist())
    new_negative = np.array(new_data[new_data["tvalues"] < 0]["roi"].tolist())

    if len(new_positive) == 0:
        warnings.warn("There are no positive values")
    if len(new_negative) == 0:
        warnings.warn("There are no negative values")

    return new_positive, new_negative

def plot_rois(r, display_mode="ortho", output_file=""):
    NODE_SIZE = 700
    fig, ax = plt.subplots(1, figsize=(8,8))
    plt.axis("off")
    plotting.plot_connectome(
        np.zeros((len(r), len(r))),
        coordinates[r-1],
        edge_threshold="95%",
        node_color=colors[func_network_affiliation[r-1]-1],
        display_mode=display_mode,
        node_size=NODE_SIZE,
        annotate=False,
        figure=fig
    )

    plt.savefig(output_file)

def plot_prob_points(*args, facecolors=None, alpha=None, capsize=10, elinewidth=10, capthick=6, dot_size=400, yticks=None, ytickslabels=None, ymin=None, ymax=None, random_state=10, zero_line=False, edge_color="none"):
    np.random.seed(random_state)
    SPREAD_LENGTH = 0.1
    
    x_list = np.arange(1.0, 1.0+0.5*len(args), 0.5)
    y_list = list(map(lambda x: x.mean(), args))
    error_list = list(map(lambda x: x.std()/np.sqrt(len(x)), args))
    spread_list = x_list - SPREAD_LENGTH/2.0
    
    # create points for means and standard errors
    fig, ax = plt.subplots(1,1, figsize=(10,10))
    ax.errorbar(x_list, y_list, error_list, ls="none", capsize=capsize, elinewidth=elinewidth, capthick=capthick, ecolor="black")
    ax.scatter(x_list, y_list, s=dot_size, c="black", zorder=10)
    
    if facecolors is None:
        facecolors = np.repeat("#cccccc", len(args))
    if alpha is None:
        alpha = np.repeat(0.5, len(args))

    if type(facecolors) == str:
        facecolors = np.repeat(facecolors, len(args))
    if type(alpha) == float or type(alpha) == int:
        alpha = np.repeat(alpha, len(args))
    
    if zero_line:
        ax.plot([1.0-0.4, max(x_list)+0.4], [0, 0], c="#cccccc", linestyle="dashed", linewidth=elinewidth, zorder=0)

    # draw each point
    for i, arg in enumerate(args):
        ax.scatter(
            np.random.rand(len(arg))*SPREAD_LENGTH + spread_list[i], 
            arg, 
            c=facecolors[i], 
            edgecolors=edge_color, 
            s=dot_size, 
            alpha=alpha[i],
            zorder=0
        )


    ax.set_xlim([1.0-0.4, max(x_list)+0.4])
    ax.set_xticks(x_list, np.repeat("", len(args)))
    ax.spines[['right', 'top']].set_visible(False)
    plt.setp(ax.spines.values(), linewidth=8)
    
    if ymin and ymax:
        ax.set_ylim([ymin, ymax])
    
    if yticks:
        if ytickslabels:
            ax.set_yticks(yticks, ytickslabels)
        else:
            ax.set_yticks(yticks)

In [ ]:
rois_positive_trans, rois_negative_trans = calc_roi_stats(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/statistics/tables_for_model/trans_data.csv")
rois_positive_alter, rois_negative_alter = calc_roi_stats(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/statistics/tables_for_model/alter_data.csv")
rois_positive_recup, rois_negative_recup = calc_roi_stats(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/statistics/tables_for_model/recup_data.csv")


In [ ]:
def bootstrap_rois(condicion: str, rois):
    condiciones = {"trans": 1, "alter": 2, "recup": 3}
    index = condiciones[condicion]

    REMOVE_REPOSO = 7-1
    REMOVE_ALTERACION = 5-1
    MAIN_DIR = "/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn"
    connectivity_regions = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300/connectivity_300.mat")["C"]
    connectivity_regions_controles = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300_controles/connectivity_300.mat")["C"]

    connectivity_regions[:,:,np.eye(300).astype(bool)] = np.nan
    connectivity_regions = np.nanmean(connectivity_regions[:,:,12:,12:], axis=3)
    connectivity_regions_condition = connectivity_regions[:, index, :]-connectivity_regions[:, 0, :]
    connectivity_regions_rois = np.nanmean(connectivity_regions_condition[:,rois], axis=1)

    connectivity_regions_controles[:,:,np.eye(300).astype(bool)] = np.nan
    connectivity_regions_controles = np.nanmean(connectivity_regions_controles[:,:,12:,12:], axis=3)
    connectivity_regions_controles[REMOVE_REPOSO, 0, :] = np.nan
    connectivity_regions_controles[REMOVE_ALTERACION, 2, :] = np.nan
    connectivity_regions_controles_condition = connectivity_regions_controles[:, index, :]-connectivity_regions_controles[:, 0, :]
    connectivity_regions_controles_rois = np.nanmean(connectivity_regions_controles_condition[:,rois], axis=1)

    N_perm = 50_000
    d_list_variacion = np.zeros(N_perm)
    for perm in tqdm(range(N_perm)):
        sampleo_controles = np.random.choice(connectivity_regions_controles_rois, 20)
        data_variacion = np.hstack((sampleo_controles, connectivity_regions_rois))
        np.random.shuffle(data_variacion)
        d_variacion = np.nanmean(data_variacion[0:20]) - np.nanmean(data_variacion[20:40])
        d_list_variacion[perm] = d_variacion

    x_variacion = np.nanmean(connectivity_regions_controles_rois) - np.nanmean(connectivity_regions_rois)

    p_variacion = np.sum(d_list_variacion > np.abs(x_variacion))/len(d_list_variacion)
    
    return p_variacion, x_variacion

In [ ]:
p1, x1 = bootstrap_rois("trans", rois_negative_trans-12)
p2, x2 = bootstrap_rois("alter", rois_positive_alter-12)
p3, x3 = bootstrap_rois("alter", rois_negative_alter-12)
p_alter = bootstrap_rois("alter", np.hstack((rois_positive_alter-12,rois_negative_alter-12)))
p4, x4 = bootstrap_rois("recup", rois_positive_recup-12)
p5, x5 = bootstrap_rois("recup", rois_negative_recup-12)
p_recup = bootstrap_rois("recup", np.hstack((rois_positive_recup-12,rois_negative_recup-12)))

p1, p2, p3, p4, p5, p_alter, p_recup

In [ ]:
p_values = np.array([p1, p2, p3, p4, p5])
h, pcorregido = fdrcorrection(p_values, alpha=0.05, method="indep", is_sorted=False)

h, pcorregido

#### bootstrap

In [ ]:
def bootstrap_red(condicion: str, network: int):
    condiciones = {"trans": 1, "alter": 2, "recup": 3}
    index = condiciones[condicion]

    REMOVE_REPOSO = 7-1
    REMOVE_ALTERACION = 5-1
    MAIN_DIR = "/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn"
    connectivity_controles = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300_controles/connectivity_2.mat")["C_net_2"]
    connectivity = sio.loadmat(f"{MAIN_DIR}/results/connectivity/300/connectivity_2.mat")["C_net_2"]

    connectivity = connectivity[:,:,0:13,0:13]
    connectivity_controles = connectivity_controles[:,:,0:13,0:13]
    connectivity[:,:,np.eye(13).astype(bool)] = np.nan
    connectivity_controles[:,:,np.eye(13).astype(bool)] = np.nan
    connectivity_controles[REMOVE_REPOSO, 0] = np.nan
    connectivity_controles[REMOVE_ALTERACION, 2] = np.nan
    connectivity_controles[REMOVE_REPOSO, 0] = np.nan
    connectivity_controles[REMOVE_ALTERACION, 2] = np.nan

    variacion = np.zeros((20,4))
    variacion_controles = np.zeros((10,4))
    for iSess in range(20):
        for iCond in range(4):
            variacion[iSess,iCond] = np.nanmean(connectivity[iSess, iCond, network, :])

    for iSess in range(10):
        for iCond in range(4):
            variacion_controles[iSess,iCond] = np.nanmean(connectivity_controles[iSess, iCond, network, :])

    variacion_diff = variacion[:,index] - variacion[:,0]
    variacion_controles_diff = variacion_controles[:,index] - variacion_controles[:,0]
    variacion_controles_diff_condicion = variacion_controles_diff[~np.isnan(variacion_controles_diff)]

    N_perm = 50_000
    d_list_variacion = np.zeros(N_perm)
    for perm in tqdm(range(N_perm)):
        sampleo_controles = np.random.choice(variacion_controles_diff_condicion, 20)
        data_variacion = np.hstack((sampleo_controles, variacion_diff))
        np.random.shuffle(data_variacion)
        d_variacion = np.nanmean(data_variacion[0:20]) - np.nanmean(data_variacion[20:40])
        d_list_variacion[perm] = d_variacion

    x_variacion = np.nanmean(variacion_controles_diff) - np.nanmean(variacion_diff)

    p_variacion = np.sum(d_list_variacion > np.abs(x_variacion))/len(d_list_variacion)
    
    return p_variacion, variacion, variacion_controles

In [ ]:
p1, _, _ = bootstrap_red("trans", network=6)
p2, _, _ = bootstrap_red("alter", network=6)
p3, _, _ = bootstrap_red("recup", network=6)

p_values = np.array([p1, p2, p3])
h, pcorregido = fdrcorrection(p_values, alpha=0.05, method="indep", is_sorted=False)

h, pcorregido

#### Plot conectividad global por red

In [ ]:
plot_prob_points(
    connectivity[:, 0, 1, :].mean(axis=1), 
    connectivity[:, 1, 1, :].mean(axis=1), 
    connectivity[:, 2, 1, :].mean(axis=1), 
    connectivity[:, 3, 1, :].mean(axis=1), 
    facecolors="#00FF00", 
    alpha=0.5,
    capsize=5, 
    elinewidth=5, 
    capthick=5,
    dot_size=500,
    zero_line=True,
    edge_color="black",
    yticks=[-0.02, 0.000, 0.02, 0.04, 0.06, 0.08, 0.10],
    ytickslabels=["-0.02", "0.00", "0.02", "0.04", "0.06", "0.08", "0.10"]
)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/avp_visual.png")

In [ ]:
plot_prob_points(
    connectivity[:, 0, 2, :].mean(axis=1), 
    connectivity[:, 1, 2, :].mean(axis=1), 
    connectivity[:, 2, 2, :].mean(axis=1), 
    connectivity[:, 3, 2, :].mean(axis=1), 
    facecolors="#0000ff", 
    alpha=0.5,
    capsize=5, 
    elinewidth=5, 
    capthick=5,
    dot_size=500,
    zero_line=True,
    edge_color="black"
)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/avp_fp.png")

In [ ]:
plot_prob_points(
    connectivity[:, 0, 6, :].mean(axis=1), 
    connectivity[:, 1, 6, :].mean(axis=1), 
    connectivity[:, 2, 6, :].mean(axis=1), 
    connectivity[:, 3, 6, :].mean(axis=1), 
    facecolors="#800080", 
    alpha=0.5,
    capsize=5, 
    elinewidth=5, 
    capthick=5,
    dot_size=500,
    zero_line=True,
    edge_color="black"
)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/avp_sn.png")

In [ ]:
plot_prob_points(
    connectivity[:, 0, 5, :].mean(axis=1), 
    connectivity[:, 1, 5, :].mean(axis=1), 
    connectivity[:, 2, 5, :].mean(axis=1), 
    connectivity[:, 3, 5, :].mean(axis=1), 
    facecolors="#00FFFF", 
    alpha=0.50,
    capsize=5, 
    elinewidth=5, 
    capthick=5,
    dot_size=500,
    zero_line=True,
    edge_color="black"
)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/avp_va.png")

In [ ]:
plot_prob_points(
    connectivity[:, 0, 8, :].mean(axis=1), 
    connectivity[:, 1, 8, :].mean(axis=1), 
    connectivity[:, 2, 8, :].mean(axis=1), 
    connectivity[:, 3, 8, :].mean(axis=1), 
    facecolors="#008000", 
    alpha=0.50,
    capsize=5, 
    elinewidth=5, 
    capthick=5,
    dot_size=500,
    zero_line=True,
    edge_color="black"
)

plt.rcParams.update({'font.size': 40})
plt.tight_layout()
plt.savefig(f"/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/avp_smd.png")

#### Plot ROIs alteracion

In [ ]:
plot_rois(rois_positive_alter, display_mode="z", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/altered_positive_z.eps")

In [ ]:
plot_rois(rois_positive_alter, display_mode="x", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/altered_positive_x.eps")

In [ ]:
plot_rois(rois_positive_alter, display_mode="y", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/altered_positive_y.eps")

In [ ]:
plot_rois(rois_negative_alter, display_mode="z", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/altered_negative_z.eps")

In [ ]:
plot_rois(rois_negative_alter, display_mode="x", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/altered_negative_x.eps")

In [ ]:
plot_rois(rois_negative_alter, display_mode="y", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/altered_negative_y.eps")

#### Plot ROIs transicion

In [ ]:
NODE_SIZE = 300
fig, ax = plt.subplots(1, figsize=(8,8))
plt.axis("off")
plotting.plot_connectome(
    np.zeros((300, 300)),
    coordinates,
    edge_threshold="95%",
    node_color="#ffffff00",
    display_mode="z",
    node_size=1,
    annotate=False,
    figure=fig
)

plt.savefig("/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/transition_positive_z.eps")

In [ ]:
NODE_SIZE = 300
fig, ax = plt.subplots(1, figsize=(8,8))
plt.axis("off")
plotting.plot_connectome(
    np.zeros((300, 300)),
    coordinates,
    edge_threshold="95%",
    node_color="#ffffff00",
    display_mode="y",
    node_size=1,
    annotate=False,
    figure=fig
)

plt.savefig("/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/transition_positive_y.eps")

In [ ]:
NODE_SIZE = 300
fig, ax = plt.subplots(1, figsize=(8,8))
plt.axis("off")
plotting.plot_connectome(
    np.zeros((300, 300)),
    coordinates,
    edge_threshold="95%",
    node_color="#ffffff00",
    display_mode="x",
    node_size=1,
    annotate=False,
    figure=fig
)

plt.savefig("/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/transition_positive_x.eps")

In [ ]:
plot_rois(rois_negative_trans, display_mode="z", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/transition_negative_z.eps")

In [ ]:
plot_rois(rois_negative_trans, display_mode="y", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/transition_negative_y.eps")

In [ ]:
plot_rois(rois_negative_trans, display_mode="x", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/transition_negative_x.eps")

#### Plot ROIs recuperacion

In [ ]:
plot_rois(rois_positive_recup, display_mode="z", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/recuperacion_positive_z.eps")

In [ ]:
plot_rois(rois_positive_recup, display_mode="y", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/recuperacion_positive_y.eps")

In [ ]:
plot_rois(rois_positive_recup, display_mode="x", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/recuperacion_positive_x.eps")

In [ ]:
plot_rois(rois_negative_recup, display_mode="z", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/recuperacion_negative_z.eps")

In [ ]:
plot_rois(rois_negative_recup, display_mode="y", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/recuperacion_negative_y.eps")

In [ ]:
plot_rois(rois_negative_recup, display_mode="x", output_file="/home/usuario/disco1/proyectos/2023-resting-state-estados-fMRI_conn/scripts/figuras-finales/conectividad-rois/figuras/recuperacion_negative_x.eps")